In [67]:
import pandas as pd
from git_root import git_root
import os


df = pd.read_json(os.path.join(git_root(),"data","pipeline_output","test_spectrogram.json"))
df['train'].head()

0    [[blues.00006.wav, [[694, 572, 485, 406, 295, ...
Name: train, dtype: object

In [68]:
label_names = set([item[3] for item in df['train'][0]])
print(label_names)

{'metal', 'reggae', 'disco', 'pop', 'country', 'blues', 'rock', 'classical', 'jazz', 'hiphop'}


In [69]:
label_to_idx = dict((name, index) for index, name in enumerate(label_names))

In [70]:
import numpy as np

samples = []
labels = []
for row in df['train'][0]:
    samples.append(np.array(row[1]))
    labels.append(label_to_idx[row[3]])

In [71]:
len(samples)

2520

In [72]:
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((samples, labels))
dataset = dataset.shuffle(128).batch(14)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [73]:
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, AveragePooling2D, Dense
from tensorflow.keras import Sequential

model = Sequential()
model.add(Conv2D(12,(6,6),activation='tanh',input_shape=(15,15,1)))
model.add(AveragePooling2D())
model.add(Conv2D(6,(3,3),activation='tanh'))
model.add(GlobalAveragePooling2D())
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 10, 10, 12)        444       
_________________________________________________________________
average_pooling2d_2 (Average (None, 5, 5, 12)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 6)           654       
_________________________________________________________________
global_average_pooling2d_2 ( (None, 6)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                70        
Total params: 1,168
Trainable params: 1,168
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [75]:
sample_batch, label_batch = next(iter(dataset))
print(sample_batch.shape)
print(label_batch.shape)

(14, 15, 15)
(14,)


In [76]:
for i in range(100):
    metrics = model.train_on_batch(tf.reshape(sample_batch,[14,15,15,1]), label_batch)
    print("Loss: {}, Accuracy: {}".format(metrics[0], metrics[1]))

Loss: 2.1887729167938232, Accuracy: 0.0
Loss: 2.064544916152954, Accuracy: 0.0
Loss: 1.9588677883148193, Accuracy: 0.0
Loss: 1.8628969192504883, Accuracy: 0.1428571492433548
Loss: 1.7887927293777466, Accuracy: 0.2857142984867096
Loss: 1.7169015407562256, Accuracy: 0.5
Loss: 1.6525064706802368, Accuracy: 0.6428571343421936
Loss: 1.595249891281128, Accuracy: 0.7857142686843872
Loss: 1.5434242486953735, Accuracy: 1.0
Loss: 1.5007869005203247, Accuracy: 1.0
Loss: 1.4609030485153198, Accuracy: 1.0
Loss: 1.4252488613128662, Accuracy: 1.0
Loss: 1.3903831243515015, Accuracy: 1.0
Loss: 1.3601140975952148, Accuracy: 1.0
Loss: 1.3324936628341675, Accuracy: 1.0
Loss: 1.3081656694412231, Accuracy: 1.0
Loss: 1.2847617864608765, Accuracy: 1.0
Loss: 1.2641582489013672, Accuracy: 1.0
Loss: 1.244765281677246, Accuracy: 1.0
Loss: 1.2271479368209839, Accuracy: 1.0
Loss: 1.2104800939559937, Accuracy: 1.0
Loss: 1.1952028274536133, Accuracy: 1.0
Loss: 1.181038498878479, Accuracy: 1.0
Loss: 1.1671264171600342

In [88]:
sample_batch, label_batch = next(iter(dataset))
print(sample_batch.reshape(256,256,1,14))

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'reshape'

In [89]:
tf.reshape(sample_batch,[256,256,1,14])

<tf.Tensor: id=2946, shape=(256, 256, 1, 14), dtype=int32, numpy=
array([[[[ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0]],

        ...,

        [[ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0]]],


       [[[ 0,  0,  0, ..., 13, 14, 24]],

        [[22,  3,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0]],

        ...,

        [[ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0]]],


       [[[ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0]],

        ...,

        [[ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0]]],


       ...,


       [[[ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  